In [1]:
import pickle
import numpy as np
from datasets import load_dataset
import tqdm
import math
import sys

In [2]:
# # GLUE
# dataset = load_dataset("glue", "sst2")
# val_dataset = dataset["validation"]

dataset = load_dataset("christophsonntag/OLID")
train_dataset = dataset["train"]
test_dataset = dataset["test"]

### CNN Based Classifier

In [218]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

In [219]:
# Define the CNN architecture for 1D vectors
class CNNClassifier(nn.Module):
    def __init__(self):
        super(CNNClassifier, self).__init__()
#         self.conv1 = nn.Conv1d(in_channels = 1, out_channels = 32, kernel_size = 3)
#         self.conv2 = nn.Conv1d(in_channels = 32, out_channels = 32, kernel_size = 3)
#         self.conv3 = nn.Conv1d(in_channels = 32, out_channels = 16, kernel_size = 3)
#         self.pool = nn.MaxPool1d(kernel_size = 2)
        self.fc1 = nn.Linear(768, 4)  # Adjust input size according to your data
        self.fc2 = nn.Linear(4, 1)      # Output layer with 1 neuron for binary classification

    def forward(self, x):
#         x = self.pool(nn.functional.relu(self.conv1(x)))
#         x = self.pool(nn.functional.relu(self.conv2(x)))
#         x = self.pool(nn.functional.relu(self.conv3(x)))
#         x = x.view(-1, 16 * 94)  # Adjust the size according to the output size of the last convolutional layer
        x = nn.functional.relu(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))  # Sigmoid activation for binary classification
        return x

In [220]:
# Sample data loader for demonstration
class SampleDataset(torch.utils.data.Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

In [221]:
# Get Training Data

# Read pkl file 
with open('embeddings/OLID-MPNET/OLID-train-768.pkl', 'rb') as f:
    vectors = pickle.load(f)
    
X = np.array(vectors)

labels = []
for item in train_dataset:
    if item['subtask_a'] == 'OFF':
        label = 0
    else:
        label = 1
    labels.append(label)
    
Y = np.array(labels)

### Determining which Subset 

In [222]:
# Read Instance Scores
with open('scores/OLID-MPNET/instance-scores-OLID-L1-30-40-30.pkl', 'rb') as f:
    instance_scores = pickle.load(f)
    
# Sort indices based on the score in descending order
def sort_indices_by_values(values):
    return sorted(range(len(values)), key=lambda i: np.abs(values[i]), reverse = True)

# Example usage:
sorted_indices = sort_indices_by_values(instance_scores)

In [275]:
# Compute subset length

percentage = 0.05
name = 'models/OLID/model-L1-5.pth' 

subset_len = int(percentage * len(X))
sorted_subset_indices = sorted_indices[:subset_len]

In [276]:
X_eff = []
Y_eff = []

for i in sorted_subset_indices:
    X_eff.append(X[i])
    Y_eff.append(Y[i])

In [277]:
N = len(X_eff)
X_train = torch.from_numpy(np.array(X_eff).reshape(N, 1, 768))
Y_train = torch.from_numpy(np.array(Y_eff).reshape(N, 1, 1)).float()
print(str(N) + ' samples selected')

662 samples selected


In [309]:
c0 = 0
c1 = 0
for i in Y_eff:
    if i == 0:
        c0 += 1
    else:
        c1 += 1
        
print('Class 0: ' + str(c0))
print('Class 1: ' + str(c1))

Class 0: 163
Class 1: 499


### Start Training

In [310]:
# Initialize the model, loss function, and optimizer
model = CNNClassifier()
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr = 0.00005)

# Create data loader
trainset = SampleDataset(X_train, Y_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size = 32, shuffle=True)

# Train the model
for epoch in tqdm.tqdm(range(200)):  # Adjust number of epochs as needed
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if i % 100 == 9:  # Print every 10 mini-batches
            print('[%d, %5d] loss: %.4f' %
                  (epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0

print('Finished Training')

 10%|█         | 21/200 [00:00<00:01, 104.34it/s]

[1,    10] loss: 0.0607
[2,    10] loss: 0.0602
[3,    10] loss: 0.0594
[4,    10] loss: 0.0603
[5,    10] loss: 0.0600
[6,    10] loss: 0.0585
[7,    10] loss: 0.0582
[8,    10] loss: 0.0601
[9,    10] loss: 0.0599
[10,    10] loss: 0.0584
[11,    10] loss: 0.0582
[12,    10] loss: 0.0594
[13,    10] loss: 0.0577
[14,    10] loss: 0.0566
[15,    10] loss: 0.0570
[16,    10] loss: 0.0570
[17,    10] loss: 0.0590
[18,    10] loss: 0.0557
[19,    10] loss: 0.0571
[20,    10] loss: 0.0562
[21,    10] loss: 0.0565
[22,    10] loss: 0.0556


 16%|█▌        | 32/200 [00:00<00:01, 98.67it/s] 

[23,    10] loss: 0.0544
[24,    10] loss: 0.0555
[25,    10] loss: 0.0548
[26,    10] loss: 0.0539
[27,    10] loss: 0.0564
[28,    10] loss: 0.0538
[29,    10] loss: 0.0545
[30,    10] loss: 0.0526
[31,    10] loss: 0.0546
[32,    10] loss: 0.0536
[33,    10] loss: 0.0524
[34,    10] loss: 0.0539
[35,    10] loss: 0.0513
[36,    10] loss: 0.0519
[37,    10] loss: 0.0525
[38,    10] loss: 0.0521
[39,    10] loss: 0.0526
[40,    10] loss: 0.0510
[41,    10] loss: 0.0510
[42,    10] loss: 0.0517


 27%|██▋       | 54/200 [00:00<00:01, 101.45it/s]

...

[143,    10] loss: 0.0346
[144,    10] loss: 0.0318
[145,    10] loss: 0.0300
[146,    10] loss: 0.0321
[147,    10] loss: 0.0320
[148,    10] loss: 0.0327
[149,    10] loss: 0.0308
[150,    10] loss: 0.0300
[151,    10] loss: 0.0335
[152,    10] loss: 0.0338
[153,    10] loss: 0.0318
[154,    10] loss: 0.0313
[155,    10] loss: 0.0345
[156,    10] loss: 0.0326
[157,    10] loss: 0.0320
[158,    10] loss: 0.0308
[159,    10] loss: 0.0345
[160,    10] loss: 0.0328
[161,    10] loss: 0.0299
[162,    10] loss: 0.0305


 86%|████████▋ | 173/200 [00:01<00:00, 98.99it/s]

[163,    10] loss: 0.0313
[164,    10] loss: 0.0293
[165,    10] loss: 0.0279
[166,    10] loss: 0.0294
[167,    10] loss: 0.0325
[168,    10] loss: 0.0287
[169,    10] loss: 0.0321
[170,    10] loss: 0.0298
[171,    10] loss: 0.0317
[172,    10] loss: 0.0290
[173,    10] loss: 0.0314
[174,    10] loss: 0.0277
[175,    10] loss: 0.0300
[176,    10] loss: 0.0297
[177,    10] loss: 0.0289
[178,    10] loss: 0.0303
[179,    10] loss: 0.0304
[180,    10] loss: 0.0306
[181,    10] loss: 0.0320
[182,    10] loss: 0.0287
[183,    10] loss: 0.0322


100%|██████████| 200/200 [00:02<00:00, 98.46it/s] 

[184,    10] loss: 0.0296
[185,    10] loss: 0.0296
[186,    10] loss: 0.0263
[187,    10] loss: 0.0284
[188,    10] loss: 0.0258
[189,    10] loss: 0.0267
[190,    10] loss: 0.0281
[191,    10] loss: 0.0291
[192,    10] loss: 0.0302
[193,    10] loss: 0.0270
[194,    10] loss: 0.0273
[195,    10] loss: 0.0238
[196,    10] loss: 0.0277
[197,    10] loss: 0.0241
[198,    10] loss: 0.0276
[199,    10] loss: 0.0288
[200,    10] loss: 0.0269
Finished Training


In [311]:
# Save the trained model
torch.save(model.state_dict(), name)

# Perform inference with the saved model
loaded_model = CNNClassifier()
loaded_model.load_state_dict(torch.load(name))
loaded_model.eval()

CNNClassifier(
  (fc1): Linear(in_features=768, out_features=4, bias=True)
  (fc2): Linear(in_features=4, out_features=1, bias=True)
)

In [312]:
# Lets do testing

# Example inference
# sample_input = torch.randn(1, 1, 768)  # Example input
# output = loaded_model(sample_input)
# print("Model output:", output.item())

# Read pkl file 
with open('embeddings/OLID-MPNET/OLID-test-768.pkl', 'rb') as f:
    test_vectors = pickle.load(f)

In [313]:
gt = []
preds = []

for i in tqdm.tqdm(range(len(test_vectors))):
    input_vector = torch.tensor(test_vectors[i].reshape(1, 1, 768))
    label = test_dataset[i]['subtask_a']
    if label == 'OFF':
        actual = 0
    else:
        actual = 1
    output = loaded_model(input_vector) 
    if output < 0.5:
        pred = 0
    else:
        pred = 1
    gt.append(actual)
    preds.append(pred)

100%|██████████| 860/860 [00:00<00:00, 10256.80it/s]


In [314]:
# Print the number of misclassifications
misc = 0
for i in range(len(gt)):
    if gt[i] != preds[i]:
        misc += 1

print(misc)

205


In [315]:
from sklearn.metrics import classification_report

# Generate classification report
report = classification_report(gt, preds)

print("Classification Report:")
print(report)

Classification Report:
              precision    recall  f1-score   support

           0       0.67      0.28      0.40       240
           1       0.77      0.95      0.85       620

    accuracy                           0.76       860
   macro avg       0.72      0.62      0.63       860
weighted avg       0.75      0.76      0.73       860

